Author: Juliette Zerick
Created: October 2018
Last modified (tested): November 3, 2018

Purpose: This notebook contains tests and output to ensure packets are routed to the proper queues, responded to appropriately, and amplified when necessary.

Current status: all tests passed

In [2]:
from inspect import getframeinfo, stack

from whisper import *
import whisper_globals as wg

'''
The test harness involves running code in an interpreter in each RPi, manually 
inserting into the message queues. Take the stimulus_response pairs, 
run stimulus in order, check against the expected response.
'''

def test_packet_routing():
    signal.signal(signal.SIGINT, signal_handler)
    
    post_t = threading.Thread(target=postal_sorter)
    post_t.daemon = True # so it does with the host process
    THREAD_BUCKET.append(post_t) # for easier cleanup
    post_t.start()

    carto_t = threading.Thread(target=cartographer)
    carto_t.daemon = True # so it does with the host process
    THREAD_BUCKET.append(carto_t) # for easier cleanup
    carto_t.start()
    
    log.info('testing packet routing')

def report(test_result):
    caller = getframeinfo(stack()[1][0])
    caller_filename = caller.filename
    caller_line_num = caller.lineno
    
    if test_result:
        log.debug('\n\ntest ok!\n')
    else:
        log.debug('failure reported by %s at line %d' % (caller_filename,caller_line_num))
    
def print_msg(M):
    print('\t to:                   %s' % (M.recipient_addr))
    print('\t saturation requested: %d' % (M.bloom_count))
    
    if M.saturation_req:
        print('\t\t initial sender:    %s' % (M.init_sender_addr)) 
        print('\t\t initial send time: %f' % (M.init_send_time))
    
    print('\t from:                 %s' % (M.sender_addr))
    print('\t time sent:            %f' % (M.send_time))
    print('\t request type:         %s' % (msg_code2msg_desc(M.msg_type)))
    print('\t payload:              %s' % (M.payload))

def fresh_test():
    mopup()
    test_packet_routing()
    wg.msg_store.reset()

def bloomed(q,sender_addr,send_time):
    q.sender_addr = sender_addr
    q.send_time = send_time
    q.bloom_count = q.bloom_count + 1
    return str(q)

def single_test(stimulus):
    global inbox_q,outbox_q,carto_q

    stim_lmsg = lora_message(stimulus)
    
    if not stim_lmsg.pkt_valid:
        log.debug('stimulus packet was invalid')
        exit()
 
    log.debug('injecting packet into the inbox')
    inbox_q.put(stim_lmsg)

    time.sleep(2)

    if stim_lmsg.hit_msg_store:
        print('\n   retrieved from message store:\n')
        
        for K in wg.msg_store.inventory.keys():
            print('     <',K,wg.msg_store.inventory[K],'>')
            retrieved_from_msg_store = wg.msg_store.stock[K]
            print_msg(retrieved_from_msg_store)
            print()
            
    if stim_lmsg.hit_outbox:
        print('\n   response retrieved from outbox:\n')
        
        retrieved_from_outbox = outbox_q.get_nowait()
        print_msg(retrieved_from_outbox)
        return retrieved_from_outbox  

In [3]:
import whisper_globals as wg

PACKET_NO_BLOOM = '%s/0/%s/%f/%d/%s//%f|'
PACKET_WITH_BLOOM = '%s/%d,%s,%f/%s/%f/%d/%s//%f|'

my_addr = MY_MAC_ADDR
neighbor_addr = 'aa:bb:cc:dd:ee:ff'
far_neighbor_addr = 'ff:ee:dd:cc:bb:aa'
multicast = '*'
init_sender_addr = '00:11:22:33:44:55'

init_send_time = now() - 20
neighbor_send_time = now() - 10
far_neighbor_send_time = now() - 5

payload = ''
response_payload = '42,42'
RSSI_val = -42

bloom_count = 1

notification = MSG_TYPE_POS_UPDATE
msg_type_request = MSG_TYPE_POS_REQUEST
msg_type_response = MSG_TYPE_POS_RESPONSE

### Scenario: MULTICAST, SATURATION REQUESTED, RESPONSE REQUESTED

In [16]:
fresh_test()

# request from the initial sender, seen by the recipients
S = PACKET_WITH_BLOOM % (multicast,
    bloom_count, init_sender_addr, init_send_time,
    init_sender_addr,init_send_time,
    msg_type_request, payload, RSSI_val)

R0 = single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:189 - continue_bloom() ] amplified (001122334455,1541278068.486482)
[whisper_globals.py:33 -  incorporate() ] incorporated (001122334455,1541278068.486482)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (001122334455,1541278068.486482) with 1 observations
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)
[whisper_globals.py:33 -  incorporate() ] incorporated (000C29530414,1541278093.122964)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (000C2953


   retrieved from message store:

     < ('001122334455', 1541278068.486482) 1 >
	 to:                   *
	 saturation requested: 2
		 initial sender:    001122334455
		 initial send time: 1541278068.486482
	 from:                 000C29530414
	 time sent:            1541278093.098591
	 request type:         MSG_TYPE_POS_REQUEST
	 payload:              

     < ('000C29530414', 1541278093.122964) 1 >
	 to:                   001122334455
	 saturation requested: 1
		 initial sender:    000C29530414
		 initial send time: 1541278093.122964
	 from:                 000C29530414
	 time sent:            1541278093.122964
	 request type:         MSG_TYPE_POS_RESPONSE
	 payload:              39.172741,-86.523283


   response retrieved from outbox:

	 to:                   001122334455
	 saturation requested: 1
		 initial sender:    000C29530414
		 initial send time: 1541278093.122964
	 from:                 000C29530414
	 time sent:            1541278093.122964
	 request type:         MSG_TYP

In [17]:
# observation of request amplified by neighbors
S = PACKET_WITH_BLOOM % (multicast,
    bloom_count+1, init_sender_addr, init_send_time,
    neighbor_addr, neighbor_send_time,
    msg_type_request, payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (001122334455,1541278068.486482) with 2 observations


In [18]:
B0 = bloomed(R0,neighbor_addr, neighbor_send_time)

single_test(B0)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (000C29530414,1541278093.122964)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (000C29530414,1541278093.122964) with 2 observations


In [19]:
B0 = bloomed(R0,far_neighbor_addr, far_neighbor_send_time)

single_test(B0)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (000C29530414,1541278093.122964)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (000C29530414,1541278093.122964) with 3 observations


In [20]:
# further observations of propagated requests
S = PACKET_WITH_BLOOM % (multicast,
    bloom_count+2, init_sender_addr, init_send_time,
    far_neighbor_addr, far_neighbor_send_time,
    msg_type_request, payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (001122334455,1541278068.486482) with 3 observations


In [21]:
# observation of neighbor's response
S = PACKET_WITH_BLOOM % (init_sender_addr,
    1, neighbor_addr, neighbor_send_time,
    neighbor_addr, neighbor_send_time,
    msg_type_response, response_payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (AABBCCDDEEFF,1541278078.486493)
[whisper.py:189 - continue_bloom() ] amplified (AABBCCDDEEFF,1541278078.486493)
[whisper_globals.py:33 -  incorporate() ] incorporated (AABBCCDDEEFF,1541278078.486493)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (AABBCCDDEEFF,1541278078.486493) with 1 observations
[whisper.py:149 - cartographer() ] received mail (AABBCCDDEEFF,1541278078.486493)


In [22]:
# observation of responses amplified by neighbors
S = PACKET_WITH_BLOOM % (init_sender_addr,
    bloom_count+1, neighbor_addr, neighbor_send_time,
    far_neighbor_addr, far_neighbor_send_time,
    msg_type_response, response_payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (AABBCCDDEEFF,1541278078.486493)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (AABBCCDDEEFF,1541278078.486493) with 2 observations


In [23]:
# observation of far neighbor's amplification of neighbor's response
S = PACKET_WITH_BLOOM % (init_sender_addr,
    bloom_count+1, neighbor_addr, neighbor_send_time,
    far_neighbor_addr, far_neighbor_send_time,
    msg_type_response, response_payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (AABBCCDDEEFF,1541278078.486493)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (AABBCCDDEEFF,1541278078.486493) with 3 observations


In [24]:
# observation of far neighbor's response
S = PACKET_WITH_BLOOM % (init_sender_addr,
    1, far_neighbor_addr, far_neighbor_send_time,
    far_neighbor_addr, far_neighbor_send_time,
    msg_type_response, response_payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (FFEEDDCCBBAA,1541278083.486502)
[whisper.py:189 - continue_bloom() ] amplified (FFEEDDCCBBAA,1541278083.486502)
[whisper_globals.py:33 -  incorporate() ] incorporated (FFEEDDCCBBAA,1541278083.486502)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (FFEEDDCCBBAA,1541278083.486502) with 1 observations
[whisper.py:149 - cartographer() ] received mail (FFEEDDCCBBAA,1541278083.486502)


In [25]:
fresh_test() 

# request propagated by neighbors
S = PACKET_WITH_BLOOM % (multicast,
    bloom_count+1, init_sender_addr, init_send_time,
    neighbor_addr, neighbor_send_time,
    msg_type_request, payload, RSSI_val)

R = single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:189 - continue_bloom() ] amplified (001122334455,1541278068.486482)
[whisper_globals.py:33 -  incorporate() ] incorporated (001122334455,1541278068.486482)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (001122334455,1541278068.486482) with 1 observations
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)
[whisper_globals.py:33 -  incorporate() ] incorporated (000C29530414,1541278115.163941)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (000C2953


   retrieved from message store:

     < ('001122334455', 1541278068.486482) 1 >
	 to:                   *
	 saturation requested: 3
		 initial sender:    001122334455
		 initial send time: 1541278068.486482
	 from:                 000C29530414
	 time sent:            1541278115.062732
	 request type:         MSG_TYPE_POS_REQUEST
	 payload:              

     < ('000C29530414', 1541278115.163941) 1 >
	 to:                   001122334455
	 saturation requested: 1
		 initial sender:    000C29530414
		 initial send time: 1541278115.163941
	 from:                 000C29530414
	 time sent:            1541278115.163941
	 request type:         MSG_TYPE_POS_RESPONSE
	 payload:              39.172741,-86.523283


   response retrieved from outbox:

	 to:                   001122334455
	 saturation requested: 1
		 initial sender:    000C29530414
		 initial send time: 1541278115.163941
	 from:                 000C29530414
	 time sent:            1541278115.163941
	 request type:         MSG_TYP

In [26]:
fresh_test() 

# responses propagated by the recipients, request unobserved
S = PACKET_WITH_BLOOM % (init_sender_addr,
    bloom_count+1, neighbor_addr, neighbor_send_time,
    far_neighbor_addr,far_neighbor_send_time,
    msg_type_response, response_payload, RSSI_val)

R = single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (AABBCCDDEEFF,1541278078.486493)
[whisper.py:189 - continue_bloom() ] amplified (AABBCCDDEEFF,1541278078.486493)
[whisper_globals.py:33 -  incorporate() ] incorporated (AABBCCDDEEFF,1541278078.486493)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (AABBCCDDEEFF,1541278078.486493) with 1 observations
[whisper.py:149 - cartographer() ] received mail (AABBCCDDEEFF,1541278078.486493)


### Scenario: NO MULTICAST, SATURATION REQUESTED, RESPONSE REQUESTED 

In [27]:
fresh_test() 

# request from the initial sender, seen by the recipient
S = PACKET_WITH_BLOOM % (my_addr,
    bloom_count, init_sender_addr, init_send_time,
    init_sender_addr,init_send_time,
    msg_type_request, payload, RSSI_val)

R0 = single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)
[whisper_globals.py:33 -  incorporate() ] incorporated (000C29530414,1541278123.037334)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (000C29530414,1541278123.037334) with 1 observations



   retrieved from message store:

     < ('000C29530414', 1541278123.037334) 1 >
	 to:                   001122334455
	 saturation requested: 1
		 initial sender:    000C29530414
		 initial send time: 1541278123.037334
	 from:                 000C29530414
	 time sent:            1541278123.037334
	 request type:         MSG_TYPE_POS_RESPONSE
	 payload:              39.172741,-86.523283


   response retrieved from outbox:

	 to:                   001122334455
	 saturation requested: 1
		 initial sender:    000C29530414
		 initial send time: 1541278123.037334
	 from:                 000C29530414
	 time sent:            1541278123.037334
	 request type:         MSG_TYPE_POS_RESPONSE
	 payload:              39.172741,-86.523283


In [28]:
# observation of a neighbor's amplification of the response
B0 = bloomed(R0,neighbor_addr, neighbor_send_time)

single_test(B0)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (000C29530414,1541278123.037334)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (000C29530414,1541278123.037334) with 2 observations


In [29]:
fresh_test() 

# observation of a request to a neighbor
S = PACKET_WITH_BLOOM % (neighbor_addr,
    bloom_count, init_sender_addr, init_send_time,
    init_sender_addr, init_send_time,
    msg_type_request, payload, RSSI_val)

single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:189 - continue_bloom() ] amplified (001122334455,1541278068.486482)
[whisper_globals.py:33 -  incorporate() ] incorporated (001122334455,1541278068.486482)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (001122334455,1541278068.486482) with 1 observations


In [30]:
# observation of the neighbor's response to the request
S = PACKET_WITH_BLOOM % (init_sender_addr,
    1, neighbor_addr, neighbor_send_time,
    neighbor_addr, neighbor_send_time,
    msg_type_response, payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (AABBCCDDEEFF,1541278078.486493)
[whisper.py:189 - continue_bloom() ] amplified (AABBCCDDEEFF,1541278078.486493)
[whisper_globals.py:33 -  incorporate() ] incorporated (AABBCCDDEEFF,1541278078.486493)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (AABBCCDDEEFF,1541278078.486493) with 1 observations
[whisper.py:149 - cartographer() ] received mail (AABBCCDDEEFF,1541278078.486493)


In [31]:
fresh_test() 

# observation of neighbor's amplification of the request
S = PACKET_WITH_BLOOM % (my_addr,
    bloom_count+3, init_sender_addr, init_send_time,
    neighbor_addr, neighbor_send_time,
    msg_type_request, response_payload, RSSI_val)

single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)
[whisper_globals.py:33 -  incorporate() ] incorporated (000C29530414,1541278142.945507)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (000C29530414,1541278142.945507) with 1 observations



   retrieved from message store:

     < ('000C29530414', 1541278142.945507) 1 >
	 to:                   001122334455
	 saturation requested: 1
		 initial sender:    000C29530414
		 initial send time: 1541278142.945507
	 from:                 000C29530414
	 time sent:            1541278142.945507
	 request type:         MSG_TYPE_POS_RESPONSE
	 payload:              39.172741,-86.523283


   response retrieved from outbox:

	 to:                   001122334455
	 saturation requested: 1
		 initial sender:    000C29530414
		 initial send time: 1541278142.945507
	 from:                 000C29530414
	 time sent:            1541278142.945507
	 request type:         MSG_TYPE_POS_RESPONSE
	 payload:              39.172741,-86.523283


### Scenario: MULTICAST, SATURATION REQUESTED, NO RESPONSE REQUESTED 

In [32]:
fresh_test() 

# notification from the initial sender, seen by the recipients
S = PACKET_WITH_BLOOM % (multicast,
    bloom_count, init_sender_addr, init_send_time,
    init_sender_addr,init_send_time,
    notification, payload, RSSI_val)

single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:189 - continue_bloom() ] amplified (001122334455,1541278068.486482)
[whisper_globals.py:33 -  incorporate() ] incorporated (001122334455,1541278068.486482)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (001122334455,1541278068.486482) with 1 observations
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)


In [33]:
# observation of the notification, amplified by neighbors
S = PACKET_WITH_BLOOM % (multicast,
    bloom_count+3, init_sender_addr, init_send_time,
    neighbor_addr,neighbor_send_time,
    notification, payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (001122334455,1541278068.486482) with 2 observations


In [34]:
fresh_test() 

# observation of the notification, amplified by neighbors
S = PACKET_WITH_BLOOM % (multicast,
    bloom_count+3, init_sender_addr, init_send_time,
    neighbor_addr,neighbor_send_time,
    notification, payload, RSSI_val)

single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:189 - continue_bloom() ] amplified (001122334455,1541278068.486482)
[whisper_globals.py:33 -  incorporate() ] incorporated (001122334455,1541278068.486482)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (001122334455,1541278068.486482) with 1 observations
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)


### Scenario: NO MULTICAST, SATURATION REQUESTED, NO RESPONSE REQUESTED 

In [35]:
fresh_test() 

# notification from the initial sender, seen by the recipient
S = PACKET_WITH_BLOOM % (my_addr,
    bloom_count, init_sender_addr, init_send_time,
    init_sender_addr,init_send_time,
    notification, payload, RSSI_val)

single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)


In [36]:
# observation of the initial request amplified by a neighbor
S = PACKET_WITH_BLOOM % (my_addr,
    bloom_count+1, init_sender_addr, init_send_time,
    neighbor_addr,neighbor_send_time,
    notification, payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)


In [37]:
# observation of message to a neighbor
S = PACKET_WITH_BLOOM % (neighbor_addr,
    bloom_count, far_neighbor_addr, far_neighbor_send_time,
    far_neighbor_addr, far_neighbor_send_time,
    notification, payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (FFEEDDCCBBAA,1541278083.486502)
[whisper.py:189 - continue_bloom() ] amplified (FFEEDDCCBBAA,1541278083.486502)
[whisper_globals.py:33 -  incorporate() ] incorporated (FFEEDDCCBBAA,1541278083.486502)
[whisper_globals.py:28 - update_inventory() ] thresholds adjusted for (FFEEDDCCBBAA,1541278083.486502) with 1 observations
[whisper.py:149 - cartographer() ] received mail (FFEEDDCCBBAA,1541278083.486502)


### Scenario: MULTICAST, SATURATION NOT REQUESTED, RESPONSE REQUESTED 

In [38]:
fresh_test() 

# request from the initial sender, seen by the recipients
S = PACKET_NO_BLOOM % (multicast,
    init_sender_addr, init_send_time,
    msg_type_request, payload, RSSI_val)

single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)



   response retrieved from outbox:

	 to:                   001122334455
	 saturation requested: 0
	 from:                 000C29530414
	 time sent:            1541278176.692764
	 request type:         MSG_TYPE_POS_RESPONSE
	 payload:              39.172741,-86.523283


In [39]:
# observation of a neighbor's response
S = PACKET_NO_BLOOM % (init_sender_addr,
    neighbor_addr, neighbor_send_time,
    msg_type_response, response_payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (AABBCCDDEEFF,1541278078.486493)
[whisper.py:149 - cartographer() ] received mail (AABBCCDDEEFF,1541278078.486493)


### Scenario: MULTICAST, SATURATION NOT REQUESTED, NO RESPONSE REQUESTED 

In [40]:
fresh_test() 

# notification from the initial sender
S = PACKET_NO_BLOOM % (multicast,
    init_sender_addr, init_send_time,
    notification, payload, RSSI_val)

single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)


### Scenario: NO MULTICAST, SATURATION NOT REQUESTED, RESPONSE REQUESTED 

In [41]:
fresh_test() 

# request from the initial sender, seen by the recipient
S = PACKET_NO_BLOOM % (my_addr,
    init_sender_addr, init_send_time,
    msg_type_request, payload, RSSI_val)

single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)



   response retrieved from outbox:

	 to:                   001122334455
	 saturation requested: 0
	 from:                 000C29530414
	 time sent:            1541278186.513560
	 request type:         MSG_TYPE_POS_RESPONSE
	 payload:              39.172741,-86.523283


In [42]:
# observation of a request made to a neighbor
S = PACKET_NO_BLOOM % (neighbor_addr,
    init_sender_addr, init_send_time,
    msg_type_request, payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)


In [43]:
# obseration of the neighbor's response to the request
S = PACKET_NO_BLOOM % (init_sender_addr,
    neighbor_addr, neighbor_send_time,
    msg_type_response, response_payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (AABBCCDDEEFF,1541278078.486493)
[whisper.py:149 - cartographer() ] received mail (AABBCCDDEEFF,1541278078.486493)


### Scenario: NO MULTICAST, SATURATION NOT REQUESTED, NO RESPONSE REQUESTED 

In [44]:
fresh_test() 

# notification directed at this device
S = PACKET_NO_BLOOM % (my_addr,
    init_sender_addr, init_send_time,
    notification, payload, RSSI_val)

single_test(S)

[whisper.py:55 -        mopup() ] cleaning up...
[whisper.py:195 - postal_sorter() ] thread active
[whisper.py:138 - cartographer() ] thread active
[<ipython-input-14-f6af153cacb2>:25 - test_packet_routing() ] testing packet routing
[whisper_globals.py:49 -        reset() ] message store reset
[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)


In [45]:
# observation of notification directed to a neighbor
S = PACKET_NO_BLOOM % (neighbor_addr,
    init_sender_addr, init_send_time,
    notification, payload, RSSI_val)

single_test(S)

[<ipython-input-14-f6af153cacb2>:70 -  single_test() ] injecting packet into the inbox
[whisper.py:210 - postal_sorter() ] got mail (001122334455,1541278068.486482)
[whisper.py:149 - cartographer() ] received mail (001122334455,1541278068.486482)
